# Exploring, Cleaning, Refining, Blending
### some good 'ol data munging

### Retrieve

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('./data/user_hits.db')

In [3]:
df_churn = pd.read_sql_query("SELECT * FROM tbl_user_hits;", conn)

### View

In [4]:
df_churn.head()

,userid,date
0,1.0,1/1/2017
1,2.0,1/2/2017
2,3.0,1/3/2017
3,4.0,1/1/2018
4,5.0,1/2/2018


In [5]:
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   userid  9 non-null      float64
 1   date    12 non-null     object 
dtypes: float64(1), object(1)
memory usage: 320.0+ bytes


In [6]:
df_churn['date'].describe()

count           12
unique           9
top       1/3/2019
freq             2
Name: date, dtype: object

### Store

In [7]:
df_churn.to_csv('./data/user_hits_export.csv')

### Restrict

In [8]:
dfr = df_churn[df_churn['userid'] == 1]
dfr.head()

,userid,date
0,1.0,1/1/2017
6,1.0,1/1/2019


### Sort

In [9]:
dfr.sort_values(by='date', ascending=False)

,userid,date
6,1.0,1/1/2019
0,1.0,1/1/2017


### Sift

In [10]:
df_cc = pd.read_csv('./data/user_hits_binning_import.csv', parse_dates=['date'])
df_cc.head()

,userid,date
0,1,2017-01-01
1,2,2017-01-02
2,3,2017-01-03
3,4,2018-01-01
4,5,2018-01-02


In [11]:
import datetime

In [12]:
now = pd.to_datetime('now')

In [13]:
df_cc['age'] = now - df_cc['date']
df_cc.head()

,userid,date,age
0,1,2017-01-01,1330 days 12:51:45.525551
1,2,2017-01-02,1329 days 12:51:45.525551
2,3,2017-01-03,1328 days 12:51:45.525551
3,4,2018-01-01,965 days 12:51:45.525551
4,5,2018-01-02,964 days 12:51:45.525551


### flipping back to uncleaned version for this section

In [14]:
pd.isnull(df_churn)

,userid,date
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,True,False


In [15]:
pd.isnull(df_churn.dropna())

,userid,date
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False


### Combine

In [16]:
df_u17 = pd.read_csv('./data/user_data_2017.csv')
df_u17.head()

,userid,date,year
0,1,1/1/2017,2017
1,2,1/2/2017,2017
2,3,1/3/2017,2017


In [17]:
df_u18 = pd.read_csv('./data/user_data_2018.csv')
df_u19 = pd.read_csv('./data/user_data_2019.csv')

In [18]:
df_comb = pd.concat([df_u17, df_u18, df_u19], ignore_index=True)
df_comb.head()

,userid,date,year
0,1,1/1/2017,2017
1,2,1/2/2017,2017
2,3,1/3/2017,2017
3,4,1/1/2018,2018
4,5,1/2/2018,2018


In [19]:
import numpy as np

In [20]:
df_cc['age'] = (datetime.datetime(2020, 2, 28) - pd.to_datetime(df_cc['date'])).dt.days

In [21]:
df_cc.head()

,userid,date,age
0,1,2017-01-01,1153
1,2,2017-01-02,1152
2,3,2017-01-03,1151
3,4,2018-01-01,788
4,5,2018-01-02,787


In [22]:
df_ages = df_cc.groupby('userid').max()
df_ages

,date,age
userid,,
1,2020-01-02,1153
2,2020-01-03,1152
3,2019-10-02,1151
4,2018-01-01,788
5,2018-01-02,787
6,2018-10-03,513
7,2019-10-03,148
8,2020-01-01,58


In [23]:
df_ages['age_bin'] = pd.cut(x=df_ages['age'], bins=[1, 365, 730, 9999], 
                            labels=['< 1 yr', '1 to 2 yrs', '3+ yrs'])
df_ages.head()

,date,age,age_bin
userid,,,
1,2020-01-02,1153,3+ yrs
2,2020-01-03,1152,3+ yrs
3,2019-10-02,1151,3+ yrs
4,2018-01-01,788,3+ yrs
5,2018-01-02,787,3+ yrs


### db ops -- joins, relationships, aggs

In [24]:
conn = sqlite3.connect('./data/user_hits_v2.db')

In [25]:
df_lj = pd.read_sql_query("SELECT u.userid, u.date, g.city, g.state FROM tbl_user_hits u left join tbl_user_geo g on u.userid = g.userid;", conn)
df_lj.head()

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,2,1/2/2019,None,None
2,3,1/3/2019,El Paso,TX


In [26]:
df_uh = pd.read_sql_query("SELECT * FROM tbl_user_hits;",conn)
df_geo = pd.read_sql_query("SELECT * FROM tbl_user_geo;", conn)

In [27]:
df_lj2 = pd.merge(left=df_uh, right=df_geo, how='left', left_on='userid',
                 right_on='userid')
df_lj2.head()

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,2,1/2/2019,NaN,NaN
2,3,1/3/2019,El Paso,TX


In [28]:
df_ugh = pd.read_sql_query("select * from tbl_user_geo_hits;", conn)
df_ugh.head()

,userid,date,city,state
0,1,1/1/2019,Dover,DE
1,3,1/1/2019,El Paso,TX
2,1,1/2/2019,Dover,DE
3,2,1/2/2019,Philadelphia,PA
4,3,1/2/2019,El Paso,TX


In [29]:
df_gbSQL = pd.read_sql_query("select city, state, count(*) as hits from tbl_user_geo_hits group by 1, 2;", conn)
df_gbSQL.head()

,city,state,hits
0,Dover,DE,3
1,El Paso,TX,2
2,Philadelphia,PA,2


In [31]:
df_gb_cs = df_ugh.groupby(["city", "state"])["userid"].count()
df_gb_cs.head()

city          state
Dover         DE       3
El Paso       TX       2
Philadelphia  PA       2
Name: userid, dtype: int64

In [32]:
conn.close()